In [1]:
import textual_help as th

/Users/hannesmoehring/.pyenv/versions/glob/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
h3d_df = th.read_data_texts("../datasets/humanml3d/texts")
kit_df = th.read_data_texts("../datasets/KIT-ML/texts")

In [3]:
h3d_df

,motion_id,description,pos_tags,num1,num2
0,011439,the person puts his hands by their mouth and t...,the/DET person/NOUN put/VERB his/DET hand/NOUN...,0.0,0.0
1,011439,person leans to the right and cups hands aroun...,person/NOUN lean/VERB to/ADP the/DET right/NOU...,0.0,0.0
2,011439,a person raised the hands and lean it into face,a/DET person/NOUN raise/VERB the/DET hand/NOUN...,0.0,0.0
3,009276,the person bend over and pick some thing up.,the/DET person/NOUN bend/VERB over/ADP and/CCO...,0.0,0.0
4,009276,"a peraon bends over, using the right leg to be...",a/DET peraon/ADJ bend/NOUN over/ADP use/VERB t...,0.0,0.0
...,...,...,...,...,...
87379,011420,a person steps back and gets ready to throw so...,a/DET person/NOUN step/VERB back/ADV and/CCONJ...,0.0,0.0
87380,011420,a person punches with their left hand.,a/DET person/NOUN punch/VERB with/ADP their/DE...,0.0,0.0
87381,M006119,a person is walking forward whilst limping wit...,a/DET person/NOUN is/AUX walk/VERB forward/ADV...,0.0,0.0
87382,M006119,person stands still then moves draging left fo...,person/NOUN stand/VERB still/ADV then/ADV move...,0.0,0.0


In [4]:
#th.analysis_routine(h3d_df)

In [5]:
th.analysis_routine(kit_df)


=== Corpus Stats ===
num_captions: 12700
num_motions: 6018
avg_tokens: 8.291968503937008
std_tokens: 4.218847656441717
median_tokens: 7.0
vocab_size: 1287
top_20_words: [('a', 14470), ('person', 8622), ('walk', 6288), ('the', 4516), ('forward', 3151), ('and', 3042), ('to', 2676), ('step', 2574), ('is', 2512), ('human', 2284)] ...
pos_distribution: {'ADJ': 6249, 'VERB': 18490, 'ADV': 9463, 'DET': 22037, 'NOUN': 27890, 'AUX': 2870, 'CCONJ': 3139, 'SCONJ': 853, 'PRON': 2347, 'ADP': 10170, 'NUM': 1088, 'PART': 279, 'PROPN': 418, 'X': 6, 'PUNCT': 2, 'INTJ': 7}
pos_means_per_caption: {'mean_ADV': 0.7451181102362204, 'mean_PRON': 0.1848031496062992, 'mean_ADJ': 0.4920472440944882, 'mean_NOUN': 2.1960629921259844, 'mean_VERB': 1.4559055118110236, 'mean_ADP': 0.8007874015748031}
captions_per_motion_mean: 2.1103356596876037
captions_per_motion_median: 2.0




=== Motion-specific Cues ===
Top verbs: [('walk', 6195), ('turn', 1210), ('push', 808), ('perform', 778), ('take', 544), ('go', 524), ('g

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.